In [2]:
pip install torchaudio soundfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 101.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 21.9 MB/s  0:00:17m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 35.7 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 22.6 MB/s  0:00:15m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 36.8 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 80.0 MB/s  0:00:00m0:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 57.4 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 40.6 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/

In [4]:
import torchaudio
import torch
import numpy as np
import json
import boto3

ENDPOINT_NAME = "urbansound-496118972"
runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

# -----------------------
# PARAMETROS DEL MODELO
# -----------------------
SAMPLE_RATE = 22050
N_MELS = 128
TARGET_FRAMES = 160


# -----------------------
# PREPROCESAMIENTO
# -----------------------
def preprocess_audio(path):
    # Cargar audio - forzar MONO
    waveform, sr = torchaudio.load(path)

    if sr != SAMPLE_RATE:
        resampler = torchaudio.transforms.Resample(sr, SAMPLE_RATE)
        waveform = resampler(waveform)

    # Convertir a 1 canal
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Extraer mel
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_mels=N_MELS,
        f_max=8000
    )

    mel = mel_spectrogram(waveform)

    # Convertir a dB
    mel_db = torchaudio.transforms.AmplitudeToDB()(mel)

    # Normalizar (simple)
    mel_db = (mel_db - mel_db.mean()) / mel_db.std()

    # Quitar el canal del audio -> (128, frames)
    mel_db = mel_db.squeeze(0)

    # Ajustar frames EXACTOS
    frames = mel_db.shape[1]

    if frames < TARGET_FRAMES:
        pad = TARGET_FRAMES - frames
        mel_db = torch.nn.functional.pad(mel_db, (0, pad))

    elif frames > TARGET_FRAMES:
        mel_db = mel_db[:, :TARGET_FRAMES]

    # Agregar canal final -> (128, 160, 1)
    mel_db = mel_db.unsqueeze(-1)

    return mel_db.numpy().astype(np.float32)


# -----------------------
# PREDICCIÓN
# -----------------------
def predict(path):
    mel = preprocess_audio(path)
    mel = mel[np.newaxis, ...]

    payload = json.dumps(mel.tolist())

    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType="application/json",
        Body=payload
    )

    raw = response["Body"].read()
    print("RAW RESPONSE:", raw)

    resp = json.loads(raw)
    preds = np.array(resp["predictions"][0])
    
    prob = float(np.max(preds))
    idx = int(np.argmax(preds))

    if prob < 0.6:
        return "indeterminado", prob

    class_names = [
        "air_conditioner","car_horn","children_playing","dog_bark","drilling",
        "engine_idling","gun_shot","jackhammer","siren","street_music"
    ]

    return class_names[idx], prob





# -----------------------
# TEST 
# -----------------------
label, prob = predict("Desert Eagle Single Shot Gunshot Sound Effect.mp3")
print("Predicción:", label, " | Prob:", prob)


RAW RESPONSE: b'{\n    "predictions": [[4.72714934e-09, 4.06661076e-12, 4.47562e-06, 0.99999547, 7.70248362e-12, 7.05242342e-13, 7.05440914e-11, 1.57621039e-16, 6.49218179e-12, 8.40275405e-09]\n    ]\n}'
Predicción: dog_bark  | Prob: 0.99999547


In [5]:
import sys
sys.executable



'/home/ec2-user/anaconda3/envs/python3/bin/python'